# Value Function Approximation

## 조건, MDP를 모를 때, 근데 MDP를 몰라도 사용가능

- table lookup 방식에서는 각 칸에 해당하는 table을 만들고 초기화해서 사용했었다. 그렇다면 현실 문제에서 state 갯수가 얼마나 되는가?
  - Backgammon, \\(10^20\\)의 states를 가진다.
  - Helicopter, 이 문제는 continuous한 state space이기 때문에 table을 만들지 못한다.
- 그렇다면 과연 이런 문제에서 어떻게 해야하는가?

## Value function approximation

- state value function \\(V(s)\\), state s 개수 만큼 table을 가지고
- action value function \\(Q(s,a)\\), 어떤 state에서 어떤 action을 했을 때 value, 모든 state-action pair만큼 table을 가져야한다.

- function approximation, 내가 봤던 state를 바탕으로 내가 안본 state도 generalize한다.(내가 안본 state에 대해서도 올바른 value를 갖도록 output을 내준다.)

$$ \hat{v}(s,w) \approx v_{\pi}(s) $$
$$ \hat{q}(s, a, w) \approx q_{\pi}(s,a)$$

- 위 \\(\hat{v}\\)은 결국 w에 대해서 학습한다는 이야기이고, \\(\hat{v}\\)은 w에 기반해서 다른 함수가 되는 것이다.

## Types of Value function approximation

<img width="506" alt="2018-11-25 10 09 54" src="https://user-images.githubusercontent.com/22078438/48974384-4cdd4700-f09a-11e8-8c0d-c417a911a2ab.png">

- \\(\hat{v}\\)은 함수에 s로 쿼리를 날리면 value가 나오는 블랙박스이다. w라는 parameter가 관련이 된다.
- \\(\hat{q}\\) 함수는 블랙박스를 두가지로 만들수 있다.
  - s와 a를 넣었을 때 value를 return 하는 방식으로 함수를 만들수도 있고,(action-in)
  - s를 넣었을 때 가능한 모든 action을 return 하는 방식으로 함수를 만들수도 있다.(action-out)
- function approximation 종류
  - Linear combinations of features
  - Neural network
  - Decision tree
  - nearest neighbour
  - Fourier
- 우리는 gradient를 update하기 위해서 미분 가능한 함수를 사용할 것이다.
- 그리고 우리는 non-stationary(모분포가 계속 바뀌고), non-iid(독립적이지 않다)

## Value function approximation by stochastic gradient descent

### 이것의 전제는 true value function \\(v_{\pi}(s)\\)를 안다는 가정이 있다.

- 목적 함수 \\(J(w)\\)를 줄이는 방향으로 w를 update해줘야한다.
$$ J(w) = \mathbb{E}_{\pi}[(v_{\pi}(S) - \hat{v}(S,w))^2] $$
- 그래서 gradient를 찾으면
$$ \begin{split} \triangle w &= -1/2\alpha \bigtriangledown_w J(w) \\
&= \alpha \mathbb{E}_{\pi}[(v_{\pi}(S) - \hat{v}(S,w)) \bigtriangledown_w \hat{v}(S,w)] \end{split} $$
  - 위에서 구한 \\(\triangle w\\)만큼 update하면 된다.
- stochasic gradient descent, \\(policy \pi\\)를 따라가면 sample(방문했던 state)이 나오는데 그것을 input으로 한다. sample로 gradient를 update하는 것이 stochasic gradient descent이다.
- 1번 sample이 많이 뽑히고, 7번 sample이 적게 뽑힌다면 결국은 자연스럽게 Expectation과 같아진다.

## Feature vectors

- state s에 대해서 사람이 임의로 feature를 만들수 있음.

## Linear Value Function Approximation

- \\(\hat{v}(S,w) = x(S)^T w = \sum^n_{j=1} x_j(S) w_j \\)
  - x1 feature에 w1곱하고, ... 그것을 모두 합하면 value이다.
  - 처음 w를 모두 0으로 초기화하면 0이된다.
- \\(J(w) = \mathbb{E}_{\pi}[(v_{\pi}(S)-x(S)^T w)^2] \\)
- 계속 w를 수정해 나가는 것이다.
- 결론적으로 식을 다 정리하면,
$$ \bigtriangledown_w \hat{v}(S,w) = x(S) $$
$$ \triangle w = \alpha(v_{\pi}(S) - \hat{v}(S,w))x(S) $$

- Linear combination일 때 위와 같이 된다.
- value function을 학습하려고 하고, 그 때 모방함수를 사용하는데 지금 모방함수를 linear combination을 사용하고 있다.

## Incremental Prediction Alogrithms

- 강화학습은 지도자가 없어서 정확한 정답을 모른다. 하지만 앞선 approximation을 배울 때는 정답을 안다고 가정하고 식을 전개 했었다. 그렇다면 정답은 어떻게 되는가?
  - 실제 값이 들어가는 \\(v_{\pi}\\)자리에 앞장에서 배운 MC와 TD를 넣을 것이다.

<img width="489" alt="2018-11-25 10 59 13" src="https://user-images.githubusercontent.com/22078438/48974633-2a9af780-f0a1-11e8-85dc-5d7b9e035194.png">


## Monte-Carlo with Value function approximation

- return \\(G_t\\)는 unbiased이다. 그 이유는 무한번 에피소드를 하다보면 결국은 true value \\(v_{\pi}(S_t)\\)에 도달한다는 것이다.
- 따라서 monte-carlo 방법을 사용하면 결국 수렴하게 되고, non-linear 할 때도 수렴한다.

## TD Learning with Value function approximation

- MC는 variance는 크지만 unbiased estimate이기 때문에 결국은 수렴하게 되는데, TD의 경우 biased이기 때문에 global optimum에 가까워진다.

<img width="378" alt="2018-11-25 11 13 02" src="https://user-images.githubusercontent.com/22078438/48974717-1952ea80-f0a3-11e8-8a53-3ebc1de7fb5a.png">

- 질문, 위 식에서 보면 TD-target을 넣을 때도 \\(\hat{v}\\)이 들어가는데 그럼 앞서 미분할 때 잘못한게 아니냐?
  - 대답, 우리는 현재 1-step 미래를 보고 예측을 하고 있는데, TD-target도 미분해버리면 미래에서 과거를 보고 과거에서 미래를 보는 두 가지 관점이 생기게 되고 이것은 너무 복잡해진다. 따라서 미래만 보고 예측을 하자.

## Control with Value Function approximation

- Policy evaluation 자리에 approximate policy evaluation을 사용한다.
- 그러면, policy를 학습할 수 있지 않냐??

- v자리에 q만 넣으면 된다.

## Batch reinforcement learning

- 지금 까지 봤던 incremental method는 gradient descent를 이용해서 sample 하나를 봤을 때 sample하나로 update하고 policy 바꾸고 번갈아 가면서 했다.
- 하지만 sample은 효과적이지 못하다. 일단 sample은 transition 1번 인데 state s에서 action a를 해서 state s'에 도착한 것 reward r을 받고 한번 update하고 그 경험은 버려지니까 sample이 효과적으로 쓰이지 않는 것
- 그래서 batch 방법은 여러번 사용을 하게 된다.

<img width="424" alt="2018-11-25 11 32 34" src="https://user-images.githubusercontent.com/22078438/48974787-d1819280-f0a5-11e8-938f-ade8dae19cd5.png">

- 데이터가 1000개가 있으면 1000개 중에 10개를 sampling해서 그것으로 update한번 하고, 또 10개 sampling해서 update하고 ...
- 그러면 한 sample이 여러번 사용된다. 데이터를 효율적으로 사용가능하다.
- **Experience Replay 방법**, off-policy 방법에서 굉장히 많이 쓰이는 방법이다.

## DQN

- experience replay
- fixed Q-targets, 예전 version이 w-, 현재 version이 w이다. 여기서 w를 update하는 것인데 td-target은 w-가 쓰여서 target을 잠깐 고정시킨다. 그 이유는 td-target이 어느 방향으로 학습시킬지 인데 update할 때 마다 방향이 계속 바뀐다. 그러면 non-linear 함수가 들어왔을 때 수렴하기가 더 힘들다.